In [1]:
import os
from pathlib import Path
from os import listdir
from os.path import isfile, join

from PIL import Image
import matplotlib.pyplot as plt

import math
import numpy as np

import tensorflow as tf
import torch
import lpips

from utils.srgan_tools import resolve_single
from utils.srgan import generator
from utils.niqe import niqe
from utils.tools import load_image, plot_sample
from skimage.metrics import structural_similarity

2022-10-04 11:24:39.722088: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
pre = generator(num_res_blocks=8, batch_size=16, return_features=True)
pre.load_weights('weights/srgan/pre_generator_small.h5')

2022-10-04 11:24:42.609486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 11:24:42.612318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 11:24:42.612406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 11:24:42.612578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [3]:
pre

In [3]:
gan = generator(num_res_blocks=8, batch_size=16, return_features=True)
gan.load_weights('weights/srgan/gan_generator_gridsearch_2022-07-26_16_43_06.2346946.h5')

In [4]:
new = generator(num_res_blocks=8, batch_size=16, return_features=True)

In [5]:
def mix_model_weights(mp, mg, p=0.5):
    
    mix_w = []
    for i in range(len(mp.layers)):
        pre_w = mp.layers[i].get_weights()
        gan_w = mg.layers[i].get_weights()
        #print(mp.layers[i].name, len(pre_w))
        
        mix_w.append(mix_layer_weights(pre_w, gan_w, p))

        
    return mix_w

In [6]:
def mix_layer_weights(wp, wg, p=0.5):
    weights = []
    for i in range(len(wp)):
        w = p * wp[i] + (1-p) * wg[i]
        weights.append(w)
    return weights

In [7]:
for p in range(0,101,10):
    mix_w = mix_model_weights(pre, gan, p/100.0)

    for i in range(len(new.layers)):
        new.layers[i].set_weights(mix_w[i])
    
    new.save_weights(f'weights/srgan/small_interpolation_{p}.h5')

In [11]:
head = generator(scale=4,
                 num_filters=64, 
                 num_res_blocks=8,
                 batch_size=16,
                 batch_norm=False,
                 activation='relu',
                 upsampling='TransposeConv',
                 head_only=True)
head.set_weights(pre.get_weights()[-4:]) 

In [21]:
for i in range(len(head.get_weights())):
    print((head.get_weights()[i] == pre.get_weights()[-4:][i]).all())

True
True
True
True


In [13]:
pre.get_weights()[-4:]

[array([[[[-0.09850702, -0.01371955,  0.07698141, ..., -0.13307124,
           -0.07525181, -0.13867883],
          [ 0.02370401,  0.05429022,  0.16665271, ..., -0.04676383,
            0.00756181, -0.01861946],
          [ 0.08967642,  0.01949177,  0.04358887, ..., -0.07308435,
            0.18989602, -0.03719832],
          ...,
          [ 0.0163812 , -0.1302753 ,  0.0272213 , ..., -0.08603533,
            0.00742552, -0.04371448],
          [-0.12349581,  0.03929252,  0.13656949, ..., -0.08429345,
           -0.05232818,  0.05681907],
          [ 0.01336618,  0.06912163,  0.15275525, ..., -0.09820163,
           -0.13328679,  0.01477259]],
 
         [[ 0.01381786, -0.13177042, -0.03978655, ...,  0.07923536,
            0.0773579 , -0.088777  ],
          [-0.13941596, -0.02247828,  0.06954904, ...,  0.02702254,
           -0.03330126, -0.04412874],
          [-0.05930866, -0.11767665, -0.03624958, ..., -0.00227606,
           -0.09965427,  0.00440576],
          ...,
          [-0